In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("kakao_listly.csv")  # kakao_listly.csv 파일을 불러오기
id_list = title_df['url'].astype(str).tolist()  # url 칼럼의 값들을 문자열로 변환하여 리스트로 저장
print(len(id_list))

# WebDriver 설정
options = Options()
options.add_argument('headless')  # 헤드리스 모드
options.add_argument('window-size=1920x1080')  # 윈도우 크기 설정
options.add_argument("disable-gpu")  # GPU 비활성화
driver = webdriver.Chrome(options=options)

# 데이터 저장 리스트
data = []

# URL 순회하며 정보 수집
for url in id_list[:5]:  # 예시로 10개만 크롤링
    # 오버뷰 페이지 URL 생성
    overview_url = f"{url}?tab_type=overview"
    driver.get(overview_url)
    time.sleep(2)  # 페이지가 완전히 로드될 시간을 줍니다.

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목
    title_h2 = soup.find("span", class_="font-large3-bold mb-3pxr text-ellipsis break-all text-el-70 line-clamp-2")
    title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"

    # 작가
    writer_span = soup.find("span", class_="font-small2 mb-6pxr text-ellipsis text-el-70 opacity-70 break-word-anywhere line-clamp-2")
    writer_text = writer_span.get_text() if writer_span else "작가를 찾을 수 없음"

    # 장르 (웹툰 제외하고 두 번째 텍스트만 추출)
    genre_spans = soup.find_all("span", class_="break-all align-middle")
    genres = [genre.get_text() for genre in genre_spans]
    
    # 웹툰은 공통이므로 첫 번째 항목은 제외하고 두 번째 장르만 추출
    if len(genres) > 1:
        genre_text = genres[1]  # 두 번째 장르만 추출
    else:
        genre_text = "장르를 찾을 수 없음"

    # 조회수
    view_count_span = soup.find("span", class_="text-el-70 opacity-70")
    view_count_text = view_count_span.get_text() if view_count_span else "조회수를 찾을 수 없음"

    # 별점
    rating_span = soup.find("span", class_="text-el-70 opacity-70")
    rating_text = rating_span.get_text() if rating_span else "별점을 찾을 수 없음"

    # 회차수
    episode_count_span = soup.find("span", class_="text-ellipsis break-all line-clamp-1 font-small2-bold text-el-70")
    episode_count_text = episode_count_span.get_text() if episode_count_span else "회차수를 찾을 수 없음"

    # 댓글수
    comment_count_span = soup.find("span", class_="text-ellipsis break-all line-clamp-1 font-small2-bold text-el-70")
    comment_count_text = comment_count_span.get_text() if comment_count_span else "댓글수를 찾을 수 없음"

    # 저장
    data.append({
        'Title': title_text,
        'Writer': writer_text,
        'Genre': genre_text,
        'View Count': view_count_text,
        'Rating': rating_text,
        'Episode Count': episode_count_text,
        'Comment Count': comment_count_text
    })

    # 10개 확인용
    print(f"현재 작품: {title_text}")
    time.sleep(3)  # 데이터 확인을 위한 시간

# 드라이버 종료
driver.quit()

# DataFrame 생성 및 출력
df = pd.DataFrame(data)[['Title', 'Writer', 'Genre', 'View Count', 'Rating', 'Episode Count', 'Comment Count']]

# CSV로 저장
df.to_csv('kakao_overview_crawling.csv', index=False, encoding='utf-8-sig')


2677
현재 작품: 나 혼자만 레벨업
현재 작품: 접근 불가 레이디
현재 작품: 제목을 찾을 수 없음
현재 작품: 무협지 악녀인데 내가 제일 쎄!
현재 작품: 그녀와 야수
